# Neural Network

In [1]:
# Import the required libraries and dependencies
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import LabelEncoder, StandardScaler

%matplotlib inline

## Using a Single Neuron for Binary Classification

### Step 1 - Import Stock and Social Media data

In [2]:
# Import stock data from .csv file
CEI_data = Path("../Resources/CEI_data.csv")
CEI_df = pd.read_csv(CEI_data, delimiter=",")
CEI_df.head()

,time,open,high,low,close,volume
0,2021-08-02 09:00:00-04:00,0.4576,0.4576,0.4566,0.4566,200
1,2021-08-02 09:15:00-04:00,0.4548,0.4548,0.4546,0.4546,200
2,2021-08-02 09:30:00-04:00,0.4514,0.4514,0.4498,0.4498,200
3,2021-08-02 09:45:00-04:00,0.4484,0.4498,0.4484,0.4498,200
4,2021-08-02 10:15:00-04:00,0.4470,0.4470,0.4470,0.4470,387


In [ ]:
# Import social media data from .csv file
SM_data = Path("../Resources/CEI_data.csv")
SM_df = pd.read_csv(CEI_data, delimiter=",")
SM_df.head()

In [ ]:
# Merge stock and social media dataframe
df = CEI_df.merge(SM_df, left_on='time', right_on='time')

### Step 2 - Encoded Data

In [ ]:
# Split target column from dataset
y = df['Close']
X = df.drop(columns='Close')

# Set Index
X = X.set_index('id')

# Binary encoding using Pandas (multiple columns) for Social Media Data
X = pd.get_dummies(X)

X.head()

### Step 3 - Scaling Data

In [ ]:
# Splitting into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Creating the scaler instance
scaler = StandardScaler()

# Fitting the scaler
X_scaler = scaler.fit(X_train)

# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Step 4 - Neural Network Model Creation

#### Add the layers of Neural Network

In [ ]:
# Create a sequential model
neuron = Sequential()

# First layer of Neural Network
number_inputs = 3
number_hidden_nodes = 6

neuron.add(Dense(units=number_hidden_nodes, activation="relu", input_dim=number_inputs))

# Create the second hidden layer
number_hidden_nodes2 = 2

neuron.add(Dense(units=number_hidden_nodes2, activation="relu"))

# Output layer
output_nodes = 1

neuron.add(Dense(units=output_nodes, activation="linear"))

# Model summary
neuron.summary()

## Compile model
## neuron.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Compile the model 
# Set the parameters as mean_squared_error, adam, and mse.
neuron.compile(loss="mean_squared_error", optimizer="adam", metrics=["mse"])

In [ ]:
# Fit the model
# The validation split parameter splits the data into testing and training sets.
model = neuron.fit(X, y, validation_split=0.3, epochs=800, verbose=0)

#### Evaluate the Model

In [ ]:
# Plotting the Loss Function and Accuracy
# Create a DataFrame with the history dictionary
df = pd.DataFrame(model.history, index=range(1, len(model.history["loss"]) + 1))

# Plot the loss
df.plot(y="loss")

In [ ]:
# Plot the accuracy
df.plot(y="accuracy")

In [ ]:
# Evaluate the model using the testing data
model_loss, model_accuracy = neuron.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

### Step 5 - Prediction

In [ ]:
# Making predictions
predictions = neuron.predict_classes(X_test_scaled)
results = pd.DataFrame({"predictions": predictions.ravel(), "actual": y_test.ravel()})
results.head(10)

In [ ]:
# Plot train vs test for the deep neural net
plt.plot(neuron.history["loss"])
plt.plot(neuron.history["val_loss"])
plt.title("loss_function - 2 hidden layers - Train Vs. Test")
plt.legend(["train", "test"])
plt.show()

In [ ]:
# Plot the CEI stock closing price Actual vs. Prediction
matplotlib.style.use('ggplot')
results.plot()
plt.xlabel("Date")
plt.ylabel("Closing Price")
plt.title("Stock Price Prediction")